In [1]:
import cv2
import numpy as np
from PIL import Image
from PIL import ImageTk
import tkinter as tk
import threading
import time

In [2]:
beta = 0.05
threshhold = 20
min_contour_area = 3000
window = None
frame_box_global = None
frame_background_global = None
frame_thresh_global = None

image_box = None
image_thresh = None
image_background = None
lock = threading.Lock()

In [3]:
camera = cv2.VideoCapture(0)
_, frame_first = camera.read()
avg = np.float32(frame_first)

In [4]:
class Running_Average_Processor(threading.Thread):
    def __init__(self, threadID, name, counter):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.counter = counter   
    def run(self):
        while(True):
            global camera, avg, min_contour_area, threshhold, beta
            _, frame_current = camera.read()
            cv2.accumulateWeighted(frame_current, avg, beta)   #Hàm để tính avg background với tham số beta

            frame_background = cv2.convertScaleAbs(avg) #Lấy background
            frame_current_gray = cv2.cvtColor(frame_current, cv2.COLOR_BGR2GRAY) 
            frame_current_gray = cv2.GaussianBlur(frame_current_gray, (21, 21), 0)       
            frame_background_gray = cv2.cvtColor(frame_background, cv2.COLOR_BGR2GRAY) 
            frame_background_gray = cv2.GaussianBlur(frame_background_gray, (21, 21), 0)   
            frame_diff = cv2.absdiff(frame_background_gray, frame_current_gray)
            frame_thresh = cv2.threshold(frame_diff, threshhold, 255, cv2.THRESH_BINARY)[1] 
            frame_thresh = cv2.dilate(frame_thresh, None, iterations = 2)  

            (_, contours, _) = cv2.findContours(frame_thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)   
            for contour in contours: 
                if cv2.contourArea(contour) > min_contour_area: 
                    (x, y, w, h) = cv2.boundingRect(contour) 
                    cv2.rectangle(frame_current, (x, y), (x + w, y + h), (0, 255, 0), 3) 

            global frame_box_global, frame_background_global, frame_thresh_global, lock
            
            lock.acquire()
            frame_thresh_global = frame_thresh
            frame_background_global = frame_background
            frame_background_global=cv2.cvtColor(frame_background_global, cv2.COLOR_BGR2RGB)     
            frame_box_global = frame_current 
            frame_box_global=cv2.cvtColor(frame_box_global, cv2.COLOR_BGR2RGB) 
            lock.release()
            
        cv2.destroyAllWindows()
        camera.release()

In [5]:
#Các hàm callback cho GUI
def set_beta(rate):
    global beta
    beta = float(rate)

def set_threshhold(rate):
    global threshhold
    threshhold = int(rate)
    
def set_contour(area):
    global min_contour_area
    min_contour_area = int(area)
    
def change_image_box():
    global frame_box_global
    if frame_box_global is None:
        return
    global window, image_box   
    photo = ImageTk.PhotoImage(Image.fromarray(frame_box_global))
    image_box.configure(image=photo)
    image_box.image = photo
    image_box.grid(row=0, column=1, rowspan = 2, columnspan = 3)
    window.after(50, change_image_box)

def change_image_thresh():
    global frame_thresh_global
    if frame_thresh_global is None:
        return
    global window, image_thresh  
    frame_thresh_global = cv2.resize(frame_thresh_global, (352,240))
    photo = ImageTk.PhotoImage(Image.fromarray(frame_thresh_global))
    image_thresh.configure(image=photo)
    image_thresh.image = photo
    image_thresh.grid(row=1, column=0)
    window.after(50, change_image_thresh)
    
def change_image_background():
    global frame_background_global
    if frame_background_global is None:
        return
    global window, image_background  
    frame_background_global = cv2.resize(frame_background_global, (352,240)) 
    photo = ImageTk.PhotoImage(Image.fromarray(frame_background_global))
    image_background.configure(image=photo)
    image_background.image = photo
    image_background.grid(row=0, column=0)
    window.after(50, change_image_background )

In [6]:
thread1 = Running_Average_Processor(1, "Thread-1", 1)
thread1.start()
time.sleep(2)

In [7]:
window = tk.Tk()
window.title('Running Average')

image_box = tk.Label(window)
image_thresh = tk.Label(window)
image_background = tk.Label(window)

scale_beta = tk.Scale(window, 
                    orient ='horizontal', 
                    from_ = 0.0, to = 0.5,
                    length = 200,
                    sliderlength = 60,
                    resolution = 0.01,
                    command = set_beta)
scale_beta.grid(row=3, column=0)
scale_beta.set(beta)

scale_thresh = tk.Scale(window, 
                        orient ='horizontal', 
                        from_ = 0, to = 255,
                        length = 200,
                        sliderlength = 60,
                        resolution = 5,
                        command = set_threshhold)
scale_thresh.grid(row=3, column=1)
scale_thresh.set(threshhold)

scale_contour = tk.Scale(window, 
                        orient ='horizontal', 
                        from_ = 0, to = 5000,
                        length = 200,
                        sliderlength = 60,
                        resolution = 50,
                        command = set_contour)
scale_contour.grid(row=3, column=2)
scale_contour.set(min_contour_area)

tk.Label(window,
        text = 'Adjust beta').grid(row=4, column=0)

tk.Label(window,
        text = 'Adjust threshhold').grid(row=4, column=1)

tk.Label(window,
        text = 'Adjust min contour area').grid(row=4, column=2)

In [8]:
window.after(50, change_image_box)
window.after(50, change_image_thresh)
window.after(50, change_image_background)
window.mainloop()